### Imports

In [1]:
%load_ext autoreload
%autoreload 2

In [4]:
import sys
from pathlib import Path

import pdblp
import numpy as np
import pandas as pd

if (path := 'C:\\dev\\projects\\credit\\backend') not in sys.path:
    sys.path.append(path)
    
from api.blp import BlpQuery
import utils

### Bloomberg Connection

In [110]:
con = pdblp.BCon(debug=False, port=8194, timeout=20000)
con.start();

In [121]:
dt_fmt = '%Y%m%d'

### Bloomberg Connection

In [5]:
bq = BlpQuery().start()

In [6]:
tickers = bq.bsrch('FI:test1')

In [171]:
isins = [
    'XS2342732562',
    'XS2461786829',
    'XS2463518998',
    'XS2463505581',
    'XS2464732770',
    'XS2462468740',
    'XS2462466611',
    'XS2466172363',
    'FR0014009L57',
    'XS2443749648',
    'XS2466172280',
    'XS2468979302',
    'XS2466171985',
    'XS2459544339',
    'XS2430287529',
    'XS2459163619',
    'XS2455983861',
    'XS2430287875',
    'XS2463918313',
    'FR0014007KL5',
    'XS2457496359',
    'XS2433361719',
    'XS2438616240',
    'XS2433135543',
    'XS2438615606',
    'XS2430287362',
    'XS2434895558',
    'XS2452435295',
    'XS2434895806',
    'XS2433244246',
    'XS2432131188',
    'XS2432293673',
    'XS2432130610',
    'XS2452434645',
    'XS2454766473',
    'XS2451372499',
    'XS2452433910',
    'XS2436160183',
    'XS2438026366',
    'FR00140082Z6',
    'XS2433244089',
    'XS2433226854',
    'XS2433211310',
    'XS2444424639',
    'XS2444263102',
    'XS2432293756',
    'XS2432293913',
    'XS2434710872',
    'XS2438026440',
    'XS2431934608',
    'XS2434710799',
    'XS2433210007',
    'XS2434895632',
    'XS2433207714',
    'XS2433208019',
    'XS2434895475',
    'XS2434393968',
    'XS2444273168',
]

In [187]:
flds = [
    'NAME',
    'SECURITY_NAME',
    'TICKER',
    'ID_ISIN',
    'ID_CUSIP',
    'TICKER_AND_EXCH_CODE',
    'EXCH_CODE',
    'MARKET_SECTOR_DES',  # yellow key
    'CRNCY',
    'COUNTRY',
    'ISSUER',
    'ISSUER_EQUITY_TICKER',
    'CNTRY_OF_RISK',
    'GICS_SECTOR',
    'GICS_INDUSTRY_GROUP',
    'GICS_INDUSTRY',
    'GICS_SUB_INDUSTRY',
    'GICS_SECTOR_NAME',
    'GICS_INDUSTRY_GROUP_NAME',
    'GICS_INDUSTRY_NAME',
    'GICS_SUB_INDUSTRY_NAME',
    'BICS_LEVEL_1_SECTOR_CODE',
    'BICS_LEVEL_2_INDUSTRY_GROUP_CODE',
    'BICS_LEVEL_1_SECTOR_NAME',
    'BICS_LEVEL_2_INDUSTRY_GROUP_NAME',
    'GREEN_BOND_LOAN_INDICATOR',
    'CPN',
    'CPN_TYP',
    'CPN_FREQ',
    'CPN_CRNCY',
    'FLT_SPREAD',
    'DAY_CNT',
    'MTY_TYP',
    'MATURITY',
    'CALC_MATURITY',
    'ISSUE_DT',
    'INT_ACC_DT',
    'SECURITY_PRICING_DATE',
    'FIRST_SETTLE_DT',
    'FIRST_CPN_DT',
    'PAYMENT_RANK',
    'BASEL_III_DESIGNATION',
    'CAPITAL_TYP_COCO_INITIAL_TRIGGER',
    'CAPITAL_TYPE_COCO_ACTION',
    'CAPITAL_TYPE_COCO_TRIGGER_LEVEL',
    'CAPITAL_TRIGGER_TYPE',
    'RTG_SP',
    'RTG_MOODY',
    'RTG_FITCH',
    'BB_COMPOSITE',
]

In [ ]:
data = 

In [188]:
data = con.ref(utils.get_isins(isins), flds)

In [189]:
pivot_data = data.pivot(index='ticker', columns='field').droplevel(0, axis=1)

In [190]:
start_date = pd.Timestamp.today() - pd.tseries.offsets.YearEnd()
end_date = pd.Timestamp.today()

In [191]:
history_flds = [
    'Z_SPRD_MID',
]

In [192]:
history_data = con.bdh(
    utils.get_isins(isins), 
    history_flds, 
    start_date.strftime(dt_fmt), 
    end_date.strftime(dt_fmt)
)

In [193]:
def get_first_valid(series):
    return series.loc[series.first_valid_index()]

def get_change(series):
    series = series.dropna()
    return series.iloc[1] - series.iloc[0]

In [194]:
merged_data = pd.concat([
    pivot_data,
    history_data.apply(get_first_valid).droplevel(1).rename('Z_SPRD_MID_0'),
], axis=1)

In [195]:
final_data = merged_data.assign(
    Z_SPRD_CHANGE=merged_data['Z_SPRD_MID'] - merged_data['Z_SPRD_MID_0']
)
final_data

AMT_OUTSTANDING BB_COMPOSITE  \
ticker                                            
/isin/FR0014007KL5     750000000.0         BBB-   
/isin/FR00140082Z6     500000000.0         BBB-   
/isin/FR0014009L57     750000000.0         BBB+   
/isin/XS2342732562    1000000000.0         BBB-   
/isin/XS2430287362     600000000.0         BBB-   
/isin/XS2430287529     500000000.0         BBB-   
/isin/XS2430287875     650000000.0         BBB-   
/isin/XS2431934608     500000000.0         BBB+   
/isin/XS2432130610    1000000000.0         BBB+   
/isin/XS2432131188     750000000.0         BBB+   
/isin/XS2432293673    1250000000.0         BBB+   
/isin/XS2432293756     750000000.0         BBB+   
/isin/XS2432293913     750000000.0         BBB+   
/isin/XS2433135543    1000000000.0         BBB-   
/isin/XS2433207714     500000000.0         BBB-   
/isin/XS2433208019     600000000.0         BBB-   
/isin/XS2433210007     650000000.0         BBB-   
/isin/XS2433211310     850000000.0          BBB   
/isin/XS2433226854     650000000.0          BBB   
/isin/XS2433244089     500000000.0          BBB   
/isin/XS2433244246     800000000.0          BBB   
/isin/XS2433361719     500000000.0         BBB-   
/isin/XS2434393968    1000000000.0         BBB-   
/isin/XS2434710799     500000000.0          BBB   
/isin/XS2434710872     500000000.0          BBB   
/isin/XS2434895475    1250000000.0         BBB-   
/isin/XS2434895558    1250000000.0         BBB-   
/isin/XS2434895632    1250000000.0         BBB-   
/isin/XS2434895806    1250000000.0         BBB-   
/isin/XS2436160183     500000000.0          BBB   
/isin/XS2438026366     575000000.0         BBB+   
/isin/XS2438026440     575000000.0         BBB+   
/isin/XS2438615606    1000000000.0         BBB+   
/isin/XS2438616240    1000000000.0         BBB+   
/isin/XS2443749648     500000000.0         BBB-   
/isin/XS2444263102     500000000.0         BBB+   
/isin/XS2444273168     800000000.0          BBB   
/isin/XS2444424639     600000000.0         BBB-   
/isin/XS2451372499     700000000.0          BBB   
/isin/XS2452433910     750000000.0          BBB   
/isin/XS2452434645     750000000.0          BBB   
/isin/XS2452435295     500000000.0          BBB   
/isin/XS2454766473    1000000000.0          BBB   
/isin/XS2455983861    1000000000.0         BBB+   
/isin/XS2457496359     500000000.0          BBB   
/isin/XS2459163619     600000000.0          BBB   
/isin/XS2459544339    1250000000.0         BBB-   
/isin/XS2461786829     600000000.0         BBB+   
/isin/XS2462466611     600000000.0         BBB+   
/isin/XS2462468740     600000000.0         BBB+   
/isin/XS2463505581     750000000.0          BBB   
/isin/XS2463518998     750000000.0          BBB   
/isin/XS2463918313     500000000.0          BBB   
/isin/XS2464732770    1000000000.0         BBB-   
/isin/XS2466171985     500000000.0         BBB+   
/isin/XS2466172280     625000000.0         BBB+   
/isin/XS2466172363     625000000.0         BBB+   
/isin/XS2468979302     500000000.0          BBB   

                             BICS_LEVEL_2_INDUSTRY_GROUP_NAME CALC_MATURITY  \
ticker                                                                        
/isin/FR0014007KL5                  Automobiles Manufacturing    2025-07-14   
/isin/FR00140082Z6                  Publishing & Broadcasting    2030-02-07   
/isin/FR0014009L57                            Food & Beverage    2029-04-07   
/isin/XS2342732562                  Automobiles Manufacturing    2170-12-28   
/isin/XS2430287362                             Internet Media    2030-01-19   
/isin/XS2430287529                             Internet Media    2026-01-19   
/isin/XS2430287875                             Internet Media    2034-01-19   
/isin/XS2431934608                  Automobiles Manufacturing    2024-01-17   
/isin/XS2432130610                            Integrated Oils    2171-04-17   
/isin/XS2432131188                            Integrated Oils    2171-01-17   
/isin/XS24

In [196]:
final_data.to_clipboard()